In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'FPL_pos', 'Price', 'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == 3]

In [7]:
df.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
57,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2083.552734,1631.614380,FWD,140.0,6.040411
85,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1843.608276,1787.674805,MID,66.0,5.879525
95,Jack Grealish,3.0,2023-24,Manchester City,Sheffield United,0.0,2083.552734,1631.614380,MID,74.0,5.777734
117,Martin Ødegaard,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,85.0,5.681315
161,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,80.0,5.369027
181,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1946.371704,1884.362427,MID,125.0,5.304678
191,Diogo Teixeira da Silva,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1946.371704,1884.362427,MID,80.0,5.275950
214,Bukayo Saka,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,86.0,5.268686
313,Kieran Trippier,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1884.362427,1946.371704,DEF,65.0,5.097010
370,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1865.608521,1677.728027,MID,85.0,4.896065


In [8]:
# df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [9]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((437, 11), (47, 11), (156, 11), (188, 11), (46, 11))

In [10]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
57,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2083.552734,1631.614380,FWD,140.0,6.040411
85,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1843.608276,1787.674805,MID,66.0,5.879525
95,Jack Grealish,3.0,2023-24,Manchester City,Sheffield United,0.0,2083.552734,1631.614380,MID,74.0,5.777734
117,Martin Ødegaard,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,85.0,5.681315
161,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,80.0,5.369027


In [11]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [12]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [13]:
smallDB = []
for i, row in df.iterrows():
    smallDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [14]:
def find_most_expensive(playersDB):
    index = 0
    price = 0
    for i, p in enumerate(playersDB):
        if p.price > price:
            index = i
            price = p.price
    return index, price
    

In [15]:
def find_N_cheapest_on_position(playersDB, n_players, position):
    output = []
    for p in playersDB:
        if p.position == position:
            if len(output) < n_players:
                output.append(p)
            else:
                i, price = find_most_expensive(output)
                if p.price < price:
                    output[i] = p
    return output

In [16]:
find_N_cheapest_on_position(playersDB, 1, "FWD")[0].__str__()

'(Folarin Balogun, FWD, 45.0, 2.07, Arsenal) - (fixtures: 1.0)'

In [17]:
# for p in playersDB:
#     print(p.__str__())

In [18]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [19]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 2, 3],
    [5, 3, 2],
    [5, 4, 1],
]

In [20]:
budget = 1000

In [21]:
def fill_bench(playersDB, formation):
    n_gk, n_def, n_mid, n_fwd = 1, 5 - formation[0], 5 - formation[1], 3 - formation[2]
    gks = find_N_cheapest_on_position(playersDB, n_gk, "GK")
    defs = find_N_cheapest_on_position(playersDB, n_def, "DEF")
    mids = find_N_cheapest_on_position(playersDB, n_mid, "MID")
    fwds = find_N_cheapest_on_position(playersDB, n_fwd, "FWD")
    print(formation, n_gk, n_def, n_mid, n_fwd)
    bench = gks + defs + mids + fwds
    for p in bench:
        print(p.__str__())
    bench_price = sum([p.price for p in bench])
    print(bench_price)
    return bench, bench_price

In [22]:
best_teams = []
for formation in possible_formations:
    bench, bench_price = fill_bench(playersDB, formation)
    best_team = best_full_teams(smallDB, [formation], budget - bench_price)

[3, 4, 3] 1 2 1 0
(Martin Dubravka, GK, 40.0, 1.23, Newcastle Utd) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 40.0, 2.58, Brighton) - (fixtures: 1.0)
(Charlie Taylor, DEF, 40.0, 1.86, Burnley) - (fixtures: 1.0)
(Mohamed Elneny, MID, 44.0, 0.0, Arsenal) - (fixtures: 1.0)
164.0
With formation [3, 4, 3]: 56.370000000000005  | (price = 803.0)
(Guglielmo Vicario, GK, 50.0, 4.17, Tottenham) - (fixtures: 1.0)
(Kieran Trippier, DEF, 65.0, 5.1, Newcastle Utd) - (fixtures: 1.0)
(Pervis Estupiñán, DEF, 52.0, 4.7, Brighton) - (fixtures: 1.0)
(Ben Chilwell, DEF, 57.0, 4.6, Chelsea) - (fixtures: 1.0)
(Solly March, MID, 66.0, 5.88, Brighton) - (fixtures: 1.0)
(Jack Grealish, MID, 74.0, 5.78, Manchester City) - (fixtures: 1.0)
(Martin Ødegaard, MID, 85.0, 5.68, Arsenal) - (fixtures: 1.0)
(Gabriel Martinelli Silva, MID, 80.0, 5.37, Arsenal) - (fixtures: 1.0)
(Erling Haaland, FWD, 140.0, 6.04, Manchester City) - (fixtures: 1.0)
(Cody Gakpo, FWD, 75.0, 4.57, Liverpool) - (fixtures: 1.0)
(Danny Welbeck, FWD, 

In [23]:
%%time
# best_team = best_full_teams(playersDB, possible_formations, budget)

CPU times: total: 0 ns
Wall time: 0 ns


In [24]:
f"Best team score: {best_team[0][1]}"

'Best team score: 56.01'

In [25]:
def best_team_to_df(best_team_squad, df, custom_fpl_order):
    names = [p.name for p in best_team_squad]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    return df[df["Name_original"].isin(names)].sort_values(by=["FPL_pos", "Price"], ascending=False)

# Define your custom order for FPL_pos
custom_fpl_order = ["FWD", "MID", "DEF", "GK"]

In [26]:
best_squad = best_team_to_df(best_team[0][2], df, custom_fpl_order)
best_squad

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
987,Guglielmo Vicario,3.0,2023-24,Tottenham,Bournemouth,0.0,1829.086060,1660.140991,GK,50.0,4.170000
724,Trent Alexander-Arnold,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1946.371704,1884.362427,DEF,80.0,4.393825
313,Kieran Trippier,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1884.362427,1946.371704,DEF,65.0,5.097010
564,Ben Chilwell,3.0,2023-24,Chelsea,Luton,1.0,1780.894531,1602.327515,DEF,57.0,4.604108
510,Pervis Estupiñán,3.0,2023-24,Brighton,West Ham,1.0,1843.608276,1787.674805,DEF,52.0,4.699287
859,Joel Matip,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1946.371704,1884.362427,DEF,50.0,4.298745
117,Martin Ødegaard,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,85.0,5.681315
161,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,80.0,5.369027
95,Jack Grealish,3.0,2023-24,Manchester City,Sheffield United,0.0,2083.552734,1631.614380,MID,74.0,5.777734
85,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1843.608276,1787.674805,MID,66.0,5.879525


In [27]:
best_squad.to_csv("predictions/best_squad.csv")